In [ ]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt

import sys
sys.path.append('../src')
import utils_explore as ue

data = ue.all_bets()

In [ ]:
df = ue.calc_win_rate(data[data.author == 'chaplygin'], 'M')
df_plot = df[['placed-date', 'count', 'success_rate']]
df_plot.plot()
df_plot.describe()

In [ ]:
df_plot_19 = df_plot[df_plot['placed-date'] >= datetime.date(2019, 1, 1)]
df_plot_19.plot()
df_plot_19.describe()